In [20]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [21]:
# mergedf = pd.read_csv('metadata_otu_merged_famID-to-str.csv')
mergedf = pd.read_csv('metadata_otu_merged_non-rarefied.csv')
# mergedf = pd.read_csv('metadata_otu_merged_famID-to-str_onlyFamwPet.csv')

mergedf.head(1)

C:\Users\Romina\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (2,3,24,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Unnamed: 0         sample_name age age_in_years age_units    agegroup  \
0           0  797.02.HA.Forehead   1            1     years  Adolescent   

  anonymized_name body_habitat  body_product    body_site  \
0           FS_01  UBERON:skin  UBERON:sebum  UBERON:face   

                                                                  ...                                                                  \
0                                                                 ...                                                                   

  k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Nocardioidaceae; g__; s__.16  \
0                                                  0                                                       

  k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__; s__.7  \
0                                                  0                                                          

  k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__[Odoribacteraceae]; g__Odoribacter; s__.1  \
0                                                  0                                                              

   k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__intermedia  \
0                                                  0                                                                  

  k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Streptococcus; s__.41  \
0                                                  0                                                          

   k__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Streptophyta; f__; g__; s__.6  \
0                                                  0                                 

  k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.54  \
0                                                  0                                                           

  k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.23  \
0                                                  0                                                          

  k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.55  \
0                                                  0                                                           

  k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum  
0                                                  0                                                                                   

[1 rows x 1088 columns]

In [22]:
print(mergedf.shape)
mergedf = mergedf.drop("Unnamed: 0", axis=1)
mergedf = mergedf[mergedf.family_relationship !="none"]
print(mergedf.shape)

(1050, 1088)
(994, 1087)


In [23]:
human_data_train = mergedf[mergedf.host_common_name =="human"]
dog_data_test = mergedf[mergedf.host_common_name =="dog"]

X_train = human_data_train.iloc[:, 53:1085]
X_test = dog_data_test.iloc[:, 53:1085]
y_train = human_data_train["familyID"].values.reshape(-1, 1)
y_test = dog_data_test["familyID"].values.reshape(-1, 1)

In [24]:
X = mergedf.iloc[:, 53:1085]
y = mergedf["familyID"].values.reshape(-1, 1)
print(X.shape, y.shape)

(994, 1032) (994, 1)


In [25]:
# X.head(1)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
# X_train.head()

In [27]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)

In [28]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

In [29]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
y_train.shape

(745, 1)

In [32]:
# model = classifier.fit(X_train, y_train.ravel())
model = classifier.fit(X_train_scaled, y_train.ravel())

In [33]:
print(f'Test Data Model Score: {round(100*(classifier.score(X_test, y_test)),2)}%')
print("------------")
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

Test Data Model Score: 18.47%
------------
First 10 Predictions:   ['family#66' 'family#42' 'family#45' 'family#42' 'family#26' 'family#35'
 'family#19' 'family#18' 'family#40' 'family#37']
First 10 Actual labels: [['family#66'], ['family#42'], ['family#45'], ['family#50'], ['family#66'], ['family#35'], ['family#63'], ['family#53'], ['family#14'], ['family#48']]


In [34]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).reset_index(drop=True).head(10)

Prediction     Actual
0  family#66  family#66
1  family#42  family#42
2  family#45  family#45
3  family#42  family#50
4  family#26  family#66
5  family#35  family#35
6  family#19  family#63
7  family#18  family#53
8  family#40  family#14
9  family#37  family#48